<a href="https://colab.research.google.com/github/ClaireChiang91/NLP_MSUcourse/blob/main/NLP_HW1_Part4_N_Gram_language_Models_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program is going to do with the assignment for the NLP course at MSU. 
https://github.com/deskool/nlp-class
Homework1 Learning Excerse 4:
https://github.com/deskool/nlp-class/blob/master/homework/HW1/assignment.ipynb

**N-grams**
An n-gram is a contiguous sequence of n items from a given sample of text or speech. Let's begin this section by defining a function that generates n-grams from our text data using the nltk library and using it to extract n-grams for n ranging from 1-5.
**The challenge**
Now that you have built your first language model, let's turn our attention back to the philosophers. For this final learning exercise, you will be building a tri-gram Nietzsche language model, and a tri-gram Russel Language Model using the 5 books from each of the philosophers you collected from Project Gutenburg earlier.

Generate a conversation between your philosophers models by seeding one of the models with a tri-gram and allow it to run until it generates a terminating character (a '.', '?' or '!'). Then, have the other model generate a response by providing the ending tri-gram of the the first model as the seed tri-gram for the other model. Iterate like this for 15 sentences and print the text.

Note: You probably learned from the earlier components of this assignment that Nietzsche and Russel have slightly different writing styles (yes, that was an understatement). For this reason, there are likely to be instances where the ending tri-gram from your Nietzsche model does not show up in the Russel language model and vice-versa. Use Laplace smoothing to handle this problem.

In [1]:
import requests
import numpy as np
import re

import nltk
from nltk.util import ngrams
#to solve the error msg: Resource punkt not found. 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Nietzsche
website = 'http://www.gutenberg.org/cache/epub/4363/pg4363.txt'
nietz    = requests.get(website).text[:200]

# Russel
website = 'http://www.gutenberg.org/cache/epub/5827/pg5827.txt'
russel = requests.get(website).text[:200]
########################################################################
#to apply the request of iterating 15 sentences
max_setence=15 
#k smoothing, which could be 0.05, 0.5 or 1; 1 is to apply the reuqest of using Laplace
k=1
#stands for n-grams
n=3

In [3]:
#define an ngram extractor
def extract_word_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [4]:
def language_model(philosopher,size,k):
  grams=extract_word_ngrams(philosopher, size)
  vocabulary = list(set(grams))
  counts = {}
  for given_word in vocabulary:
    counts[given_word] = {}
    for next_word in vocabulary:
        counts[given_word][next_word] = 0 

  for i in range(len(grams)-1):
    counts[grams[i]][grams[i+1]] += 1

  # initialize the probabilites JSON object
  probs = {}
  for given_word in vocabulary:
    probs[given_word] = {}
    for next_word in vocabulary:
      probs[given_word][next_word] = 0

  # convert the counts to probabilites
  for key, value in counts.items():
    denominator = 0
    for key2, value2 in counts[key].items():
      denominator += value2
    for key2, value2 in counts[key].items():
      #Add-k smoothing
      probs[key][key2] = (value2+k) / (denominator+k*len(vocabulary))
  return probs

In [5]:
def sample_next_gram_from_language_model(probs, given_token):
    distribution            = list(probs[given_token].values())
    sample_from_multinomial = np.random.multinomial(1,distribution)
    sample_index            = np.where(sample_from_multinomial==1)[0][0]
    word_keys               = list(probs[given_token].keys())
    next_word               = word_keys[sample_index]
    return(next_word)

In [6]:
def create_new_sentence(_probs_,seed_token):
    tokens = [seed_token]
    while any([' '.join(tokens).split(' ')[-1] not in ['.', '?','!'] ]):
      _next_word_=sample_next_gram_from_language_model(_probs_,tokens[-1])
      if all([_next_word_.split(' ')[i] not in ['.', '?','!'] for i in range(len(_next_word_.split(' ')))]):
        tokens.append(_next_word_)
      else:
        i_index=0
        while i_index < len(_next_word_.split(' ')):
          _nextword_=_next_word_.split(' ')[i_index]
          i_index+=1
          if _nextword_ not in ['.', '?','!']:
            tokens.append(_nextword_)
          else:  
            tokens.append(_nextword_)
            break
    return tokens

In [7]:
nietz_probs=language_model(nietz, n,k)
seed_token_nietz= 'Good and Evil'
' '.join(create_new_sentence(nietz_probs, seed_token_nietz))

'Good and Evil , by Friedrich Evil , by of anyone anywhere Nietzsche This eBook the use of You may copy use of anyone the use of and with almost Beyond Good and restrictions whatsoever .'

In [8]:
russel_probs=language_model(russel, n,k)
seed_token_russel= 'no cost and'

' '.join(create_new_sentence(russel_probs, seed_token_russel))

"no cost and no restrictions whatsoever use of anyone anyone anywhere at Problems of Philosophy of Philosophy , anyone anywhere at Philosophy , by is for the anyone anywhere at anywhere at no of Philosophy , the use of of Philosophy , Gutenberg 's The no restrictions whatsoever by Bertrand Russell The Problems of You may copy , by Bertrand anyone anywhere at Problems of Philosophy it , give the use of no cost and Russell This eBook is for the use of anyone \ufeffProject Gutenberg 's Problems of Philosophy Bertrand Russell This Bertrand Russell This Philosophy , by 's The Problems may copy it Gutenberg 's The is for the it , give Gutenberg 's The the use of and with almost whatsoever ."

In [11]:
def conversation(seed_p,max_setence):
  #opening
  initial_set=0
  if seed_p=='nietz':
    initial_seed='Good and Evil'
    initial_talk=' '.join(create_new_sentence(nietz_probs, initial_seed))
    print("counts:",initial_set)
    print(seed_p,"initially say:",initial_talk)
    print("=========")  
    initial_set+=1
    seed_p='russel'
  else:
    initial_seed='no cost and'
    initial_talk=' '.join(create_new_sentence(russel_probs, initial_seed))
    print("counts:",initial_set)
    print(seed_p,"initially say:",initial_talk)
    print("=========")
    initial_set+=1
    seed_p='nietz'

 #response
  while initial_set<=max_setence:
    if seed_p=='nietz':
      next_seed=' '.join(initial_talk.split(" ")[-3:])
      if next_seed in nietz_probs.keys():
        response=' '.join(create_new_sentence(nietz_probs, next_seed))
        print("counts:",initial_set)
        print(seed_p,"responds:",response)
        print("=========")
        seed_p='russel'
        initial_set+=1
      else:
        for i in range(0,len(initial_talk.split(" "))-3):
          next_seed=' '.join(initial_talk.split(" ")[(-4-i):(-1-i)])
          if next_seed in nietz_probs.keys():
            response=' '.join(create_new_sentence(nietz_probs, next_seed))
            print("counts:",initial_set)
            print(seed_p,"responds:",response)
            print("=========")
            seed_p='russel'
            initial_set+=1
          else:
            print("counts:",initial_set)
            print(seed_p,"responds: no respone")
            print("===end======")
            initial_set+=1
            seed_p='russel'
    else:
      next_seed=' '.join(initial_talk.split(" ")[-3:])
      if next_seed in russel_probs.keys():
        response=' '.join(create_new_sentence(russel_probs, next_seed))
        print("counts:",initial_set)
        print(seed_p,"responds:",response)
        print("=========")
        seed_p='nietz'
        initial_set+=1
      else:
        for i in range(0,len(initial_talk.split(" "))-3):
          next_seed=' '.join(initial_talk.split(" ")[(-4-i):(-1-i)])
          if next_seed in russel_probs.keys():
            response=' '.join(create_new_sentence(nietz_probs, next_seed))
            print(seed_p,"responds:",response)
            print("=========")
            seed_p='nietz'
            initial_set+=1
          else:
            print("counts:",initial_set)
            print(seed_p,"responds: no respone")
            print("===end======") 
            initial_set+=1
            seed_p='nietz'

In [12]:
conversation('nietz',max_setence )

counts: 0
nietz initially say: Good and Evil and with almost , by Friedrich no cost and eBook is for EBook of Beyond almost no restrictions may copy it , by Friedrich it , g of anyone anywhere Beyond Good and Good and Evil by Friedrich Nietzsche is for the anywhere at no Gutenberg EBook of EBook of Beyond EBook of Beyond Gutenberg EBook of Beyond Good and of anyone anywhere the use of Friedrich Nietzsche This and Evil , Evil , by at no cost This eBook is by Friedrich Nietzsche eBook is for use of anyone , by Friedrich restrictions whatsoever .
counts: 1
russel responds: restrictions whatsoever .
counts: 2
nietz responds: restrictions whatsoever .
counts: 3
russel responds: restrictions whatsoever .
counts: 4
nietz responds: restrictions whatsoever .
counts: 5
russel responds: restrictions whatsoever .
counts: 6
nietz responds: restrictions whatsoever .
counts: 7
russel responds: restrictions whatsoever .
counts: 8
nietz responds: restrictions whatsoever .
counts: 9
russel responds: res